In [2]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'
from tensorflow import keras
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import decode_predictions
from keras.models import Model, load_model
import keras.backend as K

import model as tcav_model
import tcav as tcav
import utils as utils
import activation_generator as act_gen
import tensorflow as tf
import utils_plot as utils_plot
import cav as cav

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from numpy.linalg import norm

import pickle

from os import listdir
from os.path import isfile, join
import operator
from PIL import Image
from keras.preprocessing import image

import os
import math

import PIL.Image
from sklearn.metrics import pairwise
import matplotlib.pyplot as plt
from keras.applications.inception_v3 import preprocess_input

from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
from scipy import stats

import tensorflow


Using TensorFlow backend.


In [3]:
K.set_learning_phase(0)
model = load_model('v3_model.h5')

/Users/tyler/anaconda3/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
sess = K.get_session()

endpoints_v3 = dict(
    input=model.inputs[0].name,
    input_tensor=model.inputs[0],
    logit=model.outputs[0].name,
    prediction=model.outputs[0].name,
    prediction_tensor=model.outputs[0],)

tf.logging.set_verbosity(0)

working_dir = '/Users/tyler/Desktop/dissertation/programming/tcav_on_azure'

label_path = os.path.join(working_dir,'labels.txt')

mymodel = tcav_model.KerasModelWrapper(sess, 
        label_path, [299, 299, 3], endpoints_v3, 
        'InceptionV3_public', (-1, 1))

In [150]:
## CAV loop
source_dir = working_dir + '/concepts/'
activation_dir =  working_dir + '/activations/'

target_class = 'zebra'
concepts = ['ladybug_sub_2', 'random500_0']

hparams = tf.contrib.training.HParams(model_type='linear', alpha=.1)
act_generator = act_gen.ImageActivationGenerator(mymodel, source_dir, activation_dir, max_examples=50)

bn_names = ['mixed0','mixed1','mixed2','mixed3','mixed4','mixed5','mixed6','mixed7','mixed8','mixed9','mixed10']
#bn_names = ['mixed8']

for bottleneck in  bn_names:
    acts = act_generator.process_and_load_activations([bottleneck], concepts + [target_class])
    cav.get_or_train_cav(concepts,bottleneck,acts,cav_hparams=hparams,cav_dir='cav_dir')


['ladybug_sub_2', 'random500_0', 'zebra']
['ladybug_sub_2', 'random500_0', 'zebra']
['ladybug_sub_2', 'random500_0', 'zebra']
['ladybug_sub_2', 'random500_0', 'zebra']
['ladybug_sub_2', 'random500_0', 'zebra']
['ladybug_sub_2', 'random500_0', 'zebra']
['ladybug_sub_2', 'random500_0', 'zebra']
['ladybug_sub_2', 'random500_0', 'zebra']
['ladybug_sub_2', 'random500_0', 'zebra']
['ladybug_sub_2', 'random500_0', 'zebra']
['ladybug_sub_2', 'random500_0', 'zebra']


In [9]:
#cav = None

In [7]:
class_1 = 'striped_sub_1'
class_2 = 'random500_3'

layer = 'mixed9'
lm_type = 'linear'
#lm_type = 'logistic'
cav_name = class_1 + '-' + class_2 + '-' + layer + '-' + lm_type + '-0.1'
sub_path = 'cavs/' + cav_name 

# Load training/testing data
path = os.path.join(working_dir,sub_path + '_test.pkl')
with open(path, 'rb') as f:
    data = pickle.load(f)
    
d = dict(data)
x_train = d['x_train']
x_test = d['x_test']
y_train = d['y_train']
y_test = d['y_test']
y_pred = d['y_pred']

path = os.path.join(working_dir,sub_path + '.pkl')

with open(path, 'rb') as f:
    cav_data = pickle.load(f)
this_cav = cav_data['cavs'][0]

FileNotFoundError: [Errno 2] No such file or directory: '/Users/tyler/Desktop/dissertation/programming/tcav_on_azure/cavs/striped_sub_1-random500_3-mixed9-linear-0.1_test.pkl'

In [ ]:
pca_components = pickle.load(open('pca/pca_components_mixed9.pkl', 'rb'))

In [ ]:
# TRAIN
x_train_embed_pca = np.dot(pca_components, x_train.T)
color_train_pca = ['red' if y == 0 else 'blue' for y in y_train]

# TEST
x_test_embed_pca = np.dot(pca_components, x_test.T)

light_blue, light_red = '#b3b8fb', '#f6889f'

color_test_pca = [light_red if y == 0 else light_blue for y in y_test]


In [ ]:
fig = plt.figure(figsize=[5,5])

plt.scatter(x_train_embed_pca[0], x_train_embed_pca[1], color = color_train_pca)
plt.scatter(x_test_embed_pca[0], x_test_embed_pca[1], color = color_test_pca) #, alpha = .6)

circ = plt.Circle((-30, 30), radius=8, edgecolor='black', facecolor='None')
ax = plt.gca()

ax.add_patch(circ)

cav_pca = np.dot(pca_components, this_cav.T)
cav_norm = vector_norm(cav_pca)
cav_norm *= 8
#origin = [-30],[30]
#plt.quiver(*origin, cav_norm[0], cav_norm[1], color='black',scale=100)

arrow_start = [-30,30]

my_arrow = plt.arrow(arrow_start[0], arrow_start[1], cav_norm[0], cav_norm[1],width=.4,
                     length_includes_head=True,head_width=2,fc='k', ec=None)
ax.add_patch(my_arrow)


plt.xlim([-40,40])
plt.ylim([-40,40])

fig_path = 'figs/jul_9/pca_' + cav_name + '.png'

path = os.path.join(working_dir,fig_path)
#plt.savefig(path,dpi=200)


In [ ]:
this_cav = -lm.coef_[0]

In [ ]:
## get original CAV
lm.fit(x_train,y_train)
original_CAV = -lm.coef_[0]

cavs = []

## remove one point
for idx, x_point in enumerate(x_train):
    if y_train[idx]==0:
        x_train_new = np.delete(x_train, idx, axis=0)
        y_train_new = np.delete(y_train, idx, axis=0)
        
        new_lm = linear_model.SGDClassifier(alpha=.1)
        new_lm.fit(x_train_new,y_train_new)
        cav_params = -new_lm.coef_[0]
        sim = euclidean_distance(vec_norm(cav_params),vec_norm(original_CAV))
        cosine_sim = cosine_similarity(cav_params,original_CAV)
        cavs.append([idx,cav_params,sim,cosine_sim,x_point])
        
cavs_sims = np.array([a[2] for a in cavs])
cos_sims = np.array([a[3] for a in cavs]) 

In [ ]:
###

In [ ]:
## Plot new
idx = 29
this_cav = cavs[idx][1]
x_point = cavs[idx][0]
x_train_new = np.delete(x_train, x_point, axis=0)
x_rem = x_train[x_point]
y_train_new = np.delete(y_train, x_point, axis=0)

x_train_embed_pca = np.dot(pca_components, x_train_new.T)
color_train_pca = ['red' if y == 0 else 'blue' for y in y_train_new]

b = np.dot(pca_components, x_rem.T)
#a = np.delete(x_train_embed_pca, x_point, axis=1)

fig = plt.figure(figsize=[5,5])
xlim = plt.xlim([-40,40])
ylim = plt.ylim([-40,40])
plt.scatter(x_train_embed_pca[0], x_train_embed_pca[1], color = color_train_pca)

plt.scatter(b[0], b[1], color = 'black')

circ = plt.Circle((-30, 30), radius=8, edgecolor='black', facecolor='None')
ax = plt.gca()
ax.add_patch(circ)
arrow_start = [-30,30]

cav_pca = np.dot(pca_components, this_cav.T)
cav_norm = vec_norm(cav_pca) * 8

my_arrow = plt.arrow(arrow_start[0], arrow_start[1], cav_norm[0], cav_norm[1],width=.4,
                     length_includes_head=True,head_width=2,fc='k', ec=None, alpha = .1)
a0 = ax.add_patch(my_arrow)

cav_pca_1 = np.dot(pca_components, original_CAV.T)
cav_norm_1 = vec_norm(cav_pca_1) * 8

my_arrow_1 = plt.arrow(arrow_start[0]+10, arrow_start[1], cav_norm_1[0], cav_norm_1[1],width=.4,
                     length_includes_head=True,head_width=2,fc='k', ec=None, alpha = .8)
print(cav_norm_1)
a1 = ax.add_patch(my_arrow_1)



In [ ]:
#len(x_train)

In [ ]:
np.argmin(cavs_sims),np.min(cavs_sims)

In [ ]:
np.argmax(cavs_sims),np.max(cavs_sims)

In [ ]:
np.argmax(cos_sims),np.max(cos_sims)

In [ ]:
np.argmin(cos_sims),np.min(cos_sims)

## TCAV score

In [14]:

#working_dir = '/home/tam/Desktop/tcav_on_azure'
working_dir = '/Users/tyler/Desktop/dissertation/programming/tcav_on_azure'

activation_dir =  working_dir + '/activations/'
cav_dir = working_dir + '/cav_dir/'
source_dir = working_dir + '/concepts/'

target = 'zebra'

random_concepts = ['random500_0','random500_1']

concepts = ['striped_sub_1']   
bottlenecks = ['mixed0']
alphas = [0.1]

In [15]:
act_generator = None
act_generator = act_gen.ImageActivationGenerator(mymodel, source_dir, activation_dir, max_examples=50)

tf.logging.set_verbosity(1)

mytcav = tcav.TCAV(sess,
        target, concepts, bottlenecks,
        act_generator, alphas,
        cav_dir = cav_dir,random_concepts = random_concepts)

results = mytcav.run(run_parallel=False)

INFO:tensorflow:mixed0 ['striped_sub_1', 'random500_0'] zebra 0.1
INFO:tensorflow:mixed0 ['striped_sub_1', 'random500_1'] zebra 0.1
INFO:tensorflow:mixed0 ['random500_0', 'random500_1'] zebra 0.1
INFO:tensorflow:mixed0 ['random500_1', 'random500_0'] zebra 0.1
INFO:tensorflow:TCAV will 4 params
INFO:tensorflow:running 4 params
INFO:tensorflow:Running param 0 of 4
INFO:tensorflow:running zebra ['striped_sub_1', 'random500_0']
['striped_sub_1', 'random500_0', 'zebra']
INFO:tensorflow:Loaded /Users/tyler/Desktop/dissertation/programming/tcav_on_azure/activations/acts_striped_sub_1_mixed0 shape (50, 35, 35, 256)
INFO:tensorflow:Loaded /Users/tyler/Desktop/dissertation/programming/tcav_on_azure/activations/acts_random500_0_mixed0 shape (50, 35, 35, 256)
INFO:tensorflow:Loaded /Users/tyler/Desktop/dissertation/programming/tcav_on_azure/activations/acts_zebra_mixed0 shape (50, 35, 35, 256)
INFO:tensorflow:CAV already exists: /Users/tyler/Desktop/dissertation/programming/tcav_on_azure/cav_dir/s

In [135]:
#results

In [192]:
class_acts.shape

(50, 8, 8, 1280)

In [191]:
this_cav.shape

(81920,)

In [290]:
bottleneck

'mixed0'

In [10]:
df = pickle.load(open('cav_dir/df.pkl', 'rb'))


In [12]:
target_class = 'zebra'
bottleneck = 'mixed0'
class_1 = 'striped_sub_1'

this_list = df[(df['target_class']==target_class) & 
                (df['bottleneck']==bottleneck) & 
                (df['class_1']==class_1)]

In [137]:
this_list

,target_class,bottleneck,class_1,class_idx,this_acc
0,zebra,mixed9,striped_sub_1,0,0.4
0,zebra,mixed9,striped_sub_1,1,0.2
0,zebra,mixed9,striped_sub_1,2,0.6
0,zebra,mixed9,striped_sub_1,3,0.4
0,zebra,mixed9,striped_sub_1,4,0.6
0,zebra,mixed9,striped_sub_1,5,0.7
0,zebra,mixed9,striped_sub_1,6,0.4
0,zebra,mixed9,striped_sub_1,7,0.4
0,zebra,mixed9,striped_sub_1,8,0.3
0,zebra,mixed9,striped_sub_1,9,0.7


In [133]:
this_cav.shape

(81920,)

In [138]:
## CAV loop
source_dir = working_dir + '/concepts/'
activation_dir =  working_dir + '/activations/'
cav_dir = working_dir + '/cav_dir'

target_class = 'zebra'
class_1 = 'striped_sub_1'
concepts = ['striped_sub_2', 'random500_3']

class_1_list = ['striped_sub_1', 'striped_sub_2']
class_1_list = ['striped_sub_1']

hparams = tf.contrib.training.HParams(model_type='linear', alpha=.1)
act_generator = act_gen.ImageActivationGenerator(mymodel, source_dir, activation_dir, max_examples=50)


bn_names = ['mixed0','mixed1','mixed2','mixed3','mixed4','mixed5','mixed6','mixed7','mixed8','mixed9','mixed10']
bn_names = ['mixed8']

tf.logging.set_verbosity(0)

columns = ['target_class','bottleneck','class_1','class_idx','this_acc']
#acc = pd.DataFrame(columns = columns)

df = pickle.load(open('cav_dir/df.pkl', 'rb'))

for class_1 in class_1_list:
    for bottleneck in  bn_names:
        for class_idx in range(50):
            # check if in df already
            if len(df[(df['target_class']==target_class) & 
                    (df['bottleneck']==bottleneck) & 
                    (df['class_1']==class_1) &
                    (df['class_idx']==class_idx)]) == 0:
                print(f'running {target_class}, {bottleneck}, {class_1}, {class_idx}')

                concepts = [class_1, 'random500_' + str(class_idx)]
                acts = act_generator.process_and_load_activations([bottleneck], concepts + [target_class])
                cav_data = cav.get_or_train_cav(concepts,bottleneck,acts,cav_hparams=hparams,cav_dir=cav_dir)
                this_cav = cav_data.cavs[0]

                class_acts = acts[target_class][bottleneck]
                this_acc = comp_tcav(target_class,this_cav,class_acts,bottleneck)

                this_dict = {'target_class':target_class,
                             'bottleneck':bottleneck,
                             'class_1':class_1,
                             'class_idx':class_idx,
                             'this_acc':this_acc
                            }
                df = df.append([this_dict])
            else:
                print(f'already in df {target_class}, {bottleneck}, {class_1}, {class_idx}')
            
pickle.dump(df,open('cav_dir/df.pkl', 'wb'))
print('### did write ###')

already in df zebra, mixed8, striped_sub_1, 0
already in df zebra, mixed8, striped_sub_1, 1
already in df zebra, mixed8, striped_sub_1, 2
already in df zebra, mixed8, striped_sub_1, 3
already in df zebra, mixed8, striped_sub_1, 4
already in df zebra, mixed8, striped_sub_1, 5
already in df zebra, mixed8, striped_sub_1, 6
already in df zebra, mixed8, striped_sub_1, 7
already in df zebra, mixed8, striped_sub_1, 8
already in df zebra, mixed8, striped_sub_1, 9
already in df zebra, mixed8, striped_sub_1, 10
already in df zebra, mixed8, striped_sub_1, 11
already in df zebra, mixed8, striped_sub_1, 12
already in df zebra, mixed8, striped_sub_1, 13
already in df zebra, mixed8, striped_sub_1, 14
already in df zebra, mixed8, striped_sub_1, 15
already in df zebra, mixed8, striped_sub_1, 16
already in df zebra, mixed8, striped_sub_1, 17
already in df zebra, mixed8, striped_sub_1, 18
already in df zebra, mixed8, striped_sub_1, 19
already in df zebra, mixed8, striped_sub_1, 20
already in df zebra, mi

In [129]:
bottleneck = 'mixed9'
class_1 = 'striped_sub_1'

this_list = df[(df['target_class']==target_class) & 
                (df['bottleneck']==bottleneck) & 
                (df['class_1']==class_1)]

In [130]:
this_list

,target_class,bottleneck,class_1,class_idx,this_acc
0,zebra,mixed9,striped_sub_1,0,0.4
0,zebra,mixed9,striped_sub_1,1,0.2
0,zebra,mixed9,striped_sub_1,2,0.6
0,zebra,mixed9,striped_sub_1,3,0.4
0,zebra,mixed9,striped_sub_1,4,0.6
0,zebra,mixed9,striped_sub_1,5,0.7
0,zebra,mixed9,striped_sub_1,6,0.4
0,zebra,mixed9,striped_sub_1,7,0.4
0,zebra,mixed9,striped_sub_1,8,0.3
0,zebra,mixed9,striped_sub_1,9,0.7


In [120]:
this_list['this_acc'].mean()

0.4739999999999999

In [102]:
run_t_test(this_list['this_acc'])
#acc_list

Ttest_1sampResult(statistic=-2.2204296750073302, pvalue=0.05352649820286758)

In [123]:
bn_names = ['mixed0','mixed1','mixed2','mixed3','mixed4','mixed5','mixed6','mixed7','mixed8','mixed9','mixed10']

stats_dict = {}
for bn in bn_names:
    target_class = 'zebra'
    this_list = df[(df['target_class']==target_class) & 
                (df['bottleneck']==bn)]
    stats_dict[bn] = this_list.mean()

In [124]:
stats_dict

{'mixed0': this_acc    0.605
 dtype: float64, 'mixed1': this_acc    0.526
 dtype: float64, 'mixed2': this_acc    0.474
 dtype: float64, 'mixed3': this_acc    0.54
 dtype: float64, 'mixed4': this_acc    0.56
 dtype: float64, 'mixed5': this_acc    0.51425
 dtype: float64, 'mixed6': this_acc    0.76
 dtype: float64, 'mixed7': this_acc    0.418
 dtype: float64, 'mixed8': this_acc    0.645
 dtype: float64, 'mixed9': this_acc    0.42
 dtype: float64, 'mixed10': this_acc    1.0
 dtype: float64}

In [ ]:
## Get list of TCAV scores
tcavs = []

class_1 = 'knitted_sub_2'
class_2 = 'random500_0'

layer = 'mixed9'
lm_type = 'linear'
cav_name = class_1 + '-' + class_2 + '-' + layer + '-' + lm_type + '-0.1'
sub_path = 'cavs/' + cav_name 

path = os.path.join(working_dir,sub_path + '.pkl')

with open(path, 'rb') as f:
    cav_data = pickle.load(f)
this_cav = cav_data['cavs'][0]

# GET ACTS
bottleneck_name = layer
acts = act_generator.process_and_load_activations([bottleneck_name], concepts + [target_class])
class_acts = acts[target_class][layer]

In [113]:
# GET CAV
class_1 = 'knitted_sub_2'
class_2 = 'random500_0'

layer = 'mixed9'
lm_type = 'linear'
cav_name = class_1 + '-' + class_2 + '-' + layer + '-' + lm_type + '-0.1'
sub_path = 'cavs/' + cav_name 

path = os.path.join(working_dir,sub_path + '.pkl')

with open(path, 'rb') as f:
    cav_data = pickle.load(f)
this_cav = cav_data['cavs'][0]

# GET ACTS
bottleneck_name = layer
acts = act_generator.process_and_load_activations([bottleneck_name], concepts + [target_class])
class_acts = acts[target_class][layer]

['knitted_sub_2', 'zebra']
INFO:tensorflow:Loaded /home/tyler/Desktop/tcav_on_azure/activations/acts_knitted_sub_2_mixed9 shape (50, 8, 8, 2048)
INFO:tensorflow:Loaded /home/tyler/Desktop/tcav_on_azure/activations/acts_zebra_mixed9 shape (50, 8, 8, 2048)


In [200]:
comp_tcav(target_class,this_cav,class_acts,'mixed8')

0.84

In [199]:
this_cav.shape

(81920,)

## t-test

In [35]:

a = np.array([45,45,50,1])
a = np.random.rand(100) * 5 + 47
stats.ttest_1samp(a,50)

def run_t_test(this_list):
    return stats.ttest_1samp(this_list,.5)

In [19]:
def comp_tcav(target_class,cav,class_acts,bottleneck_name):
    count = 0
    class_id = mymodel.label_to_id(target_class)

    for i in range(len(class_acts)):
        act = np.expand_dims(class_acts[i], 0)
        if get_dir(act, class_id,cav,bottleneck_name):
            count += 1
    return float(count) / float(len(class_acts))

def get_dir(acts, class_id, this_cav, bottleneck_name):
    grads = sess.run(mymodel.bottlenecks_gradients[bottleneck_name], 
                 {mymodel.bottlenecks_tensors[bottleneck_name]: acts,
                 mymodel.y_input: [class_id]})
                     
    grad = np.reshape(grads, -1)
    dot_prod = np.dot(grad, this_cav)

    return dot_prod < 0

In [ ]:
working_dir = '/home/tyler/Desktop/tcav_on_azure'
activation_dir =  working_dir + '/activations/'
cav_dir = working_dir + '/cavs/'
source_dir = working_dir + '/concepts/'

target = 'zebra'

concepts = ['banded_sub_1','banded_sub_2']   

#'mixed0','mixed1', 'mixed2', 'mixed3', 'mixed4', 'mixed5', 'mixed6', 'mixed7', 'mixed8', 'mixed9_0', 'mixed9', 'mixed9_1', 'mixed10'
bottlenecks = ['mixed9']

alphas = [0.1]


In [ ]:
act_generator = act_gen.ImageActivationGenerator(mymodel, source_dir, activation_dir, max_examples=50)

tf.logging.set_verbosity(0)

mytcav = tcav.TCAV(sess,
        target, concepts, bottlenecks,
        act_generator, alphas,
        cav_dir = cav_dir,
        random_concepts=['random500_3'])

In [ ]:
mytcav.run()

In [ ]:
mytcav.cav_dir

In [ ]:
import run_params

In [ ]:
these_params = run_params.RunParams(bottleneck, concepts, target,
                                   act_generator, cav_dir,
                                   alphas[0], mymodel)

In [ ]:
cav_data

In [ ]:
class_acts = acts[target_class][cav_data['bottleneck']]

comp_tcav(mymodel,target,concepts[0],cav_data,class_acts)

In [ ]:
mymodel.y_input


In [ ]:
cav_data['bottleneck']

In [ ]:
bottlenecks_tensors = get_bottleneck_tensors()

In [ ]:
#bottlenecks_tensors

In [ ]:
ends = dict(
    input=model.inputs[0].name,
    input_tensor=model.inputs[0],
    logit=model.outputs[0].name,
    prediction=model.outputs[0].name,
    prediction_tensor=model.outputs[0],)

bottlenecks_tensors = get_bottleneck_tensors()

graph = tf.get_default_graph()

# Construct gradient ops.
with graph.as_default():
    y_input = tf.placeholder(tf.int64, shape=[None])

    pred = tf.expand_dims(ends['prediction_tensor'][0], 0)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits_v2(
          labels=tf.one_hot(y_input,ends['prediction_tensor'].get_shape().as_list()[1]),logits=pred))
    #make_gradient_tensors()
    bottlenecks_gradients = {}
    for bn in bottlenecks_tensors:
        bottlenecks_gradients[bn] = tf.gradients(loss, bottlenecks_tensors[bn])[0]
    
def make_gradient_tensors():
    bottlenecks_gradients = {}
    for bn in bottlenecks_tensors:
        bottlenecks_gradients[bn] = tf.gradients(loss, bottlenecks_tensors[bn])[0]

In [ ]:
ends

In [ ]:
tf.gradients(loss, bottlenecks_tensors['mixed9'])

In [ ]:
y = mymodel.label_to_id(target_class)

In [ ]:
sess.run()

In [ ]:
#mymodel.get_gradient(acts,y,'mixed9')

In [ ]:
#bottlenecks_gradients['mixed9']

In [ ]:
#bottlenecks_gradients

In [ ]:
bottlenecks_gradients['mixed9']

In [ ]:
bottleneck_name = 'mixed9'
#y = 
grads = sess.run(bottlenecks_gradients[bottleneck_name], feed_dict = {bottlenecks_tensors[bottleneck_name]: acts, y_input: y})

In [ ]:
#bottlenecks_tensors

In [ ]:


def get_bottleneck_tensors():
    """Add Inception bottlenecks and their pre-Relu versions to endpoints dict."""
    graph = tf.get_default_graph()
    bn_endpoints = {}
    for op in graph.get_operations():
        # change this below string to change which layers are considered bottlenecks
        # use 'ConcatV2' for InceptionV3
        # use 'MaxPool' for VGG16 (for example)
        if 'ConcatV2' in op.type:
            name = op.name.split('/')[0]
            bn_endpoints[name] = op.outputs[0]
    return bn_endpoints




In [ ]:
#mytcav._run_single_set(these_params)

In [ ]:
  def get_key(self):
    return '_'.join([
        str(self.bottleneck), '_'.join(self.concepts),
        'target_' + self.target_class, 'alpha_' + str(self.alpha)
    ])

## CAVs using center of mass

In [ ]:
class_1 = 'striped_sub_1'
class_2 = 'random500_3'

layer = 'mixed9'
lm_type = 'linear'
#lm_type = 'logistic'
cav_name = class_1 + '-' + class_2 + '-' + layer + '-' + lm_type + '-0.1'
sub_path = 'cavs/' + cav_name 

# Load training/testing data
path = os.path.join(working_dir,sub_path + '_test.pkl')
with open(path, 'rb') as f:
    data = pickle.load(f)
    
d = dict(data)
x_train = d['x_train']
x_test = d['x_test']
y_train = d['y_train']
y_test = d['y_test']
y_pred = d['y_pred']

path = os.path.join(working_dir,sub_path + '.pkl')

with open(path, 'rb') as f:
    cav_data = pickle.load(f)
this_cav = cav_data['cavs'][0]

In [ ]:
x_train_p_list,x_train_n_list = [],[]
for idx,a in enumerate(x_train):
    if y_train[idx] == 0:
        x_train_p_list.append(a)
    else:
        x_train_n_list.append(a)
x_train_p,x_train_n = np.array(x_train_p_list),np.array(x_train_n_list)


In [ ]:
#x_train.shape

In [ ]:
p_avg = x_train_p.mean(axis=0,keepdims=True)
n_avg = x_train_n.mean(axis=0,keepdims=True)
cav_by_mass = (p_avg - n_avg).reshape(-1)

In [ ]:
fig = plt.figure(figsize=[5,5])

# TRAIN
x_train_embed_pca = np.dot(pca_components, x_train.T)
color_train_pca = ['red' if y == 0 else 'blue' for y in y_train]

plt.scatter(x_train_embed_pca[0], x_train_embed_pca[1], color = color_train_pca)
#plt.scatter(x_test_embed_pca[0], x_test_embed_pca[1], color = color_test_pca) #, alpha = .6)

circ = plt.Circle((-30, 30), radius=8, edgecolor='black', facecolor='None')
ax = plt.gca()
ax.add_patch(circ)

cav_pca = np.dot(pca_components, this_cav.T)
cav_norm = vec_norm(cav_pca) * 8
arrow_start = [-30,30]
my_arrow = plt.arrow(arrow_start[0], arrow_start[1], cav_norm[0], cav_norm[1],width=.4,
                     length_includes_head=True,head_width=2,fc='k', ec=None,alpha=.1)
ax.add_patch(my_arrow)

cav_pca = np.dot(pca_components, cav_by_mass.T)
cav_norm = vec_norm(cav_pca) * 8
my_arrow = plt.arrow(arrow_start[0], arrow_start[1], cav_norm[0], cav_norm[1],width=.4,
                     length_includes_head=True,head_width=2,fc='r', ec='r')
ax.add_patch(my_arrow)


plt.xlim([-40,40])
plt.ylim([-40,40])

fig_path = 'figs/jul_9/pca_' + cav_name + '.png'

path = os.path.join(working_dir,fig_path)
#plt.savefig(path,dpi=200)


In [ ]:
this_cav.shape

In [ ]:
cav_by_mass.shape

In [ ]:
cosine_similarity(this_cav,cav_by_mass)

## Using Random activations

In [ ]:
d_acts = 131072
n_acts = 33
random_gen = np.random.normal(loc = 0, scale = 1, size = (n_acts,d_acts))
x_train_n = np.where(random_gen>0,random_gen,0)

In [ ]:
p_avg = x_train_p.mean(axis=0,keepdims=True)
n_avg = x_train_n.mean(axis=0,keepdims=True)
cav_by_mass = (p_avg - n_avg).reshape(-1)

In [ ]:
fig = plt.figure(figsize=[5,5])

# Postive
x_train_embed_pca = np.dot(pca_components, x_train_p.T)
color_train_pca = 'red'
plt.scatter(x_train_embed_pca[0], x_train_embed_pca[1], color = color_train_pca)

# Negative
x_train_embed_pca = np.dot(pca_components, x_train_n.T)
color_train_pca = 'blue'
plt.scatter(x_train_embed_pca[0], x_train_embed_pca[1], color = color_train_pca)

#circ = plt.Circle((-30, 30), radius=8, edgecolor='black', facecolor='None')
ax = plt.gca()
#ax.add_patch(circ)

cav_pca = np.dot(pca_components, this_cav.T)
cav_norm = vector_norm(cav_pca) * 8
arrow_start = [-30,30]
my_arrow = plt.arrow(arrow_start[0], arrow_start[1], cav_norm[0], cav_norm[1],width=.4,
                     length_includes_head=True,head_width=2,fc='k', ec=None,alpha=.1)
ax.add_patch(my_arrow)

cav_pca = np.dot(pca_components, cav_by_mass.T)
cav_norm = vector_norm(cav_pca) * 8
my_arrow = plt.arrow(arrow_start[0], arrow_start[1], cav_norm[0], cav_norm[1],width=.4,
                     length_includes_head=True,head_width=2,fc='r', ec='r')
ax.add_patch(my_arrow)
plt.xlim([-40,40])
plt.ylim([-40,40])



In [139]:
## TCAV Experiment 1

In [ ]:
## CAV loop
source_dir = working_dir + '/concepts/'
activation_dir =  working_dir + '/activations/'
cav_dir = working_dir + '/cav_dir'

target_class = 'zebra_50'
class_1 = 'striped_sub_1'

class_1_list = ['striped_sub_1']

hparams = tf.contrib.training.HParams(model_type='linear', alpha=.1)

bn_names = ['mixed0','mixed1','mixed2','mixed3','mixed4','mixed5','mixed6','mixed7','mixed8','mixed9','mixed10']
bn_names = ['mixed8']

tf.logging.set_verbosity(0)

columns = ['target_class','bottleneck','class_1','class_idx','this_acc']
#acc = pd.DataFrame(columns = columns)

df = pickle.load(open('cav_dir/df.pkl', 'rb'))

for class_1 in class_1_list:
    for bottleneck in  bn_names:
        for class_idx in range(50):
            # check if in df already
            if len(df[(df['target_class']==target_class) & 
                    (df['bottleneck']==bottleneck) & 
                    (df['class_1']==class_1) &
                    (df['class_idx']==class_idx)]) == 0:
                print(f'running {target_class}, {bottleneck}, {class_1}, {class_idx}')

                concepts = [class_1, 'random500_' + str(class_idx)]
                acts = act_generator.process_and_load_activations([bottleneck], concepts + [target_class])
                cav_data = cav.get_or_train_cav(concepts,bottleneck,acts,cav_hparams=hparams,cav_dir=cav_dir)
                this_cav = cav_data.cavs[0]

                class_acts = acts[target_class][bottleneck]
                this_acc = comp_tcav(target_class,this_cav,class_acts,bottleneck)

                this_dict = {'target_class':target_class,
                             'bottleneck':bottleneck,
                             'class_1':class_1,
                             'class_idx':class_idx,
                             'this_acc':this_acc
                            }
                df = df.append([this_dict])
            else:
                print(f'already in df {target_class}, {bottleneck}, {class_1}, {class_idx}')
            
pickle.dump(df,open('cav_dir/df.pkl', 'wb'))
print('### did write ###')

In [ ]:
## define functions
def vector_norm(vec):
    norm_const = math.sqrt(vec[0]*vec[0] + vec[1]*vec[1])
    return vec / norm_const

def vec_norm(vec):
    return  vec / np.linalg.norm(vec)

## CAV distance
def euclidean_distance(a,b):
    return np.linalg.norm(a-b)

def cosine_similarity(v1,v2):
    # compute cosine similarity of v1 to v2: (v1 dot v2)/{||v1||*||v2||)
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    return sumxy / math.sqrt(sumxx*sumyy)